In [2]:
import nltk
import numpy as np
from scipy.stats import pearsonr
import sim_cal

[nltk_data] Downloading package stopwords to
[nltk_data]     D:/misc/Projects/Python/NLP/misc...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     D:/misc/Projects/Python/NLP/misc...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     D:/misc/Projects/Python/NLP/misc...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     D:/misc/Projects/Python/NLP/misc...
[nltk_data]   Unzipping corpora\genesis.zip.


In [4]:
with open('msr_paraphrase_corpus.txt','r') as f:
    t=f.readlines()

In [5]:
t

['Quality\tID1\tID2\tString1\tString2\n',
 '1\t702876\t702977\tAmrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.\tReferring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.\n',
 "0\t2108705\t2108831\tYucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion.\tYucaipa bought Dominick's in 1995 for $693 million and sold it to Safeway for $1.8 billion in 1998.\n",
 "1\t1330381\t1330521\tThey had published an advertisement on the Internet on June 10, offering the cargo for sale, he added.\tOn June 10, the ship's owners had published an advertisement on the Internet, offering the explosives for sale.\n",
 '0\t3344667\t3344648\tAround 0335 GMT, Tab shares were up 19 cents, or 4.4%, at A$4.56, having earlier set a record high of A$4.57.\tTab shares jumped 20 cents, or 4.6%, to set a record closing high at A$4.57.\n',
 '1\t1236820\t1236712\tThe stock rose $2.11, or abou